# Object Detection On Images using YOLO-V3

### Step 1:
1. Import cv2 and numpy libraries. 
2. Then we load yolo v3 algorithm using cv2.dnn.readNet by passing weights and cfg file. 
3. Then we will load all classes names in array using coco.names file.

In [4]:
import cv2

In [5]:
import numpy as np

In [ ]:
#Load YOLO
net = cv2.dnn.readNet("yolov3.weights","yolov3.cfg")
classes = []
with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]

#### These are the 80 labels or categories YOLO V3 can identify.

In [ ]:
classes

## Step 2:
- Next we will define output layers because that’s where we will be defining what object is detected by using __net.getUnconnectedOutLayers__ and __net.getLayerNames__.

- __getUnconnectedOutLayers() __: gives the names of the unconnected output layers, which are essentially the last layers of the network. 
- Then we run the forward pass of the network to get output from the output layers, as you will see  __(net.forward(getOutputsNames(net)))__.

In [ ]:
net.getLayerNames()

In [ ]:
net.getUnconnectedOutLayers()

In [ ]:
layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [ ]:
outputlayers

In [ ]:
colors= np.random.uniform(0,255,size=(len(classes),3))

### Step 3:
- Next let us load an image. We will reduce the height and width of our image to scale of 40% and 30%. And save all those values in height,width,channels variables for theoriginal image.

In [ ]:
#loading image
img = cv2.imread("demo1.jpg")
img = cv2.resize(img,None,fx=0.4,fy=0.3)
height,width,channels = img.shape

- Let's view the image

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# To know the shape of images
def image_and_shapes(image):
    img= plt.imread(image)
    plt.imshow(img)
    print("Shape of the image:{}".format(img.shape))

image_and_shapes("demo1.jpg")

- This is our original image from which we want to detect as many objects as possible. But we cannot give this image directly to algorithm.So we need to do some conversion from this image. This is called blob conversion which is basically extracting features from image.

- __We will detect objects in blob by using cv2.dnn.blobFromImage and passing few variables: img is file name, scalefactor of 0.00392, size of image to be used in blob be (416,416), no mean subtraction from layers as (0,0,0), setting True flag means we will be inverting blue with red since OpenCV uses BGR but we have channels in image as RGB.__

### cv.dnn.blobFromImage(img, scale, size, mean)
- __blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)__
- the image to transform
- the scale factor (1/255 to scale the pixel values to [0..1])
- the size, here a 416x416 square image
- the mean value (default=0)
- the option swapBR=True (since OpenCV uses BGR)

In [ ]:
#detecting objects
blob = cv2.dnn.blobFromImage(img,0.00392,(416,416),(0,0,0),True,crop=False)

- We now pass this blob to network using __net.setInput(blob)__ and then forward this to the __outputlayers__. Here all objects have been detected and outs contains all the information we need to instruct to extract the position of the object like top,left,right,bottom positions,name of class.

In [ ]:
net.setInput(blob)
outs = net.forward(outputlayers)

In [ ]:
print(outs[1])

### Let's evaluate outs by showing information on screen.

In [ ]:
#Showing info on screen/ get confidence score of algorithm in detecting an object in blob
class_ids=[]
confidences=[]
boxes=[]
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        
        
        if confidence > 0.5:
            #onject detected
            center_x= int(detection[0]*width)
            center_y= int(detection[1]*height)
            w = int(detection[2]*width)
            h = int(detection[3]*height)
        
            #cv2.circle(img,(center_x,center_y),10,(0,255,0),2)
            #rectangle co-ordinaters
            x=int(center_x - w/2)
            y=int(center_y - h/2)
            #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            
            boxes.append([x,y,w,h]) #put all rectangle areas
            confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
            class_ids.append(class_id) #name of the object tha was detected

In [ ]:
confidences

In [ ]:
class_ids

- There might be cases that multiple time the same object might be detected like below.  We want to eliminate this.
- To eliminate this, we will use Non-Max Suppression(NMS) functionality. What this will do is eliminate the boxes by using some threshold value(any box having value less than 0.6- that will be removed) and it determines that keep only the best of all boxes. And indexes variable will keep track of such unique objects detected. So no multiple detection of same objects.

In [ ]:
#cv2.dnn.NMSBoxes(boxes,confidences,score_threshold,IOU_threshold)
indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)#Non Max Suppressions

'''Now using below loop over all found boxes, 
if box is appearing in indexes then only draw rectangle, color it,
put text of class name on it.'''
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x,y,w,h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
        cv2.putText(img,label,(x,y+30),font,1,(255,255,255),2)
            
cv2.imshow("Image",img)
cv2.waitKey(0)
cv2.destroyAllWindows()